In [ ]:
from rxn4chemistry import RXN4ChemistryWrapper as r4c #REST API wrapper
from rdkit import Chem,RDLogger
from rdkit.Chem import AllChem, MACCSkeys, rdChemReactions as Reactions
from time import sleep #to prevent RXN cloud overload
import os
import pandas as pd
import math

In [ ]:
#Disable RDKit warning 'not removing hydrogen atom without neighbors' 
RDLogger.DisableLog('rdApp.*')

In [ ]:
#api = 'uncomment this cell and paste your API key here'

In [ ]:
def get_reactions():
    """This function collects all reaction SMILES from the file 'test-set.xlsx.'
    """
    directory = os.getcwd()
    data = (pd.read_excel(directory+'/test-set.xlsx'))
    keys = list(data.keys())
    vals = [[rxn.split('>>') for rxn in data[key]] for key in keys]
    canon = [['>>'.join([Chem.MolToSmiles(Chem.MolFromSmiles(mol)) for mol in rxn]) for rxn in rxns] for rxns in vals]
    return {keys[i]: canon[i] for i in range(len(keys))}

In [ ]:
def get_mols(m):
    """This function collects either the reactants (r) or products (p) 
    and saves them to a dictionary divided by reaction class.
    """
    if m == 'r':
        i = 0
    elif m == 'p': 
        i = -1
    else: 
        return 'Incorrect molecule description'        
    return {key : [p.split('>')[i] for p in get_reactions()[key]] for key in get_reactions().keys()}

In [ ]:
#uncomment this cell to initialize prediction output file
#head = ['reaction class','projectId','predictionId','reaction','product','confidence']
#output_headers = pd.DataFrame([],columns = [h for h in head])
#output_headers.to_csv(os.getcwd()+'/results.csv',mode = 'w')

In [ ]:
def synthesis(rxn_class):
    """
    This function performs reaction predictions per reaction class and writes the data to the file 'results.csv.'
    
    A dictionary for with reaction class, projectId, predictionId, reaction, product, and confidence is create.
    After logging in to the IBM RXN cloud with your authentication key generated on the RXN web interface under 
    the 'My Profile' tab, a project is created. A reaction prediction is then performed for each reaction,
    maintained by the 'While' loop to wait for the prediction to be complete before attempting to access the results.
    The sleep functions throughout the function serve to prevent the REST API from receiving too many requests 
    per 2 seconds and per minute in accordance with the RXN for Chemistry guidelines.
    """
    print('\n Reactions Completed: ',end = ' ')
    out = {'reaction class':[],'projectId':[],'predictionId':[],'reaction':[],'product':[],'confidence':[]}
    ibm = r4c(api_key = api)
    sleep(2)
    response = ibm.create_project(rxn_class)
    sleep(2)
    for i,rxn in enumerate(get_mols('r')[rxn_class]):
        sleep(2)
        response = ibm.predict_reaction(rxn) 
        sleep(2)
        while True:#close loop only when synthesis is complete
            #run synthesis
            results = ibm.get_predict_reaction_results(response['prediction_id'])
            #pause for 2 sec to avoid overloading IBM-RXN
            sleep(2)
            #if synthesis is complete,...
            if results['response']['payload']['status'] == 'SUCCESS':
                sleep(60)
                final = results['response']['payload']
                smiles = final['attempts'][0]['smiles']
                out['reaction class'].append(rxn_class)
                out['projectId'].append(final['projectId'])
                out['predictionId'].append(final['attempts'][0]['predictionId'])
                out['reaction'].append(get_reactions()[rxn_class][i])
                out['product'].append(smiles.split('>')[-1])
                out['confidence'].append(final['attempts'][0]['confidence']) 
                print(str(i+1), end = ' ')
                break
            else:
                results = ibm.get_predict_reaction_results(response['prediction_id'])
                sleep(60)
    output = pd.DataFrame([[out['reaction class'][j],out['projectId'][j],
                            out['predictionId'][j],out['reaction'][j],
                            out['product'][j],out['confidence'][j]] for j in range(len(out['product']))])
    output.to_csv(os.getcwd()+'/results.csv',mode='a',header=False)
    

In [ ]:
#uncomment this cell to run all predictions
#for rxn in rxnClass:
    #synthesis(rxn)

In [ ]:
#import initial prediction results and all outcomes generated from this notebook and "extractOutcomes.nb", respectively
results = pd.read_csv(os.getcwd()+'/results.csv')
more_outcomes = pd.read_csv(os.getcwd()+"/more_outcomes.csv")

In [ ]:
#get list of reacton classes
#necessary to loop over reaction classes
rxnClass = list(get_reactions().keys())

In [ ]:
#generate a dictionary of success and confidence per reaction class
success = [[more_outcomes['success'][i] for i in range(len(more_outcomes['success'])) if more_outcomes['reaction'][i] == rxn] for rxn in results['reaction'].tolist()]
split_success = [success[i : i+10] for i in range(0,len(success),10)]
conf = [[more_outcomes['confidence'][i] for i in range(len(more_outcomes['confidence'])) if more_outcomes['reaction'][i] == rxn] for rxn in results['reaction'].tolist()]
split_conf = [conf[i : i+10] for i in range(0,len(conf),10)]
outcome_dict = {rxnClass[i] : {'success':split_success[i],'confidence': split_conf[i]} for i in range(len(rxnClass))}

In [ ]:
def top_k(rxn_class):
    """
    This function computes the top-k accuracy per reaction class.
    
    Since each reaction class contains exactly 10 reactions, 
    the top-k accuracy is represented as a whole number (out of 10) rather than a percentage.
    """
    top = [0,0,0,0,0]
    for k in range(1,6):
        top_n = 0
        success = [rxn[:k] for rxn in outcome_dict[rxn_class]['success']]
        for rxn in success:
            if 'Y' in rxn:
                top[k-1] +=1
    return top

In [ ]:
#generate dictionary of top-k accuracies divided by reaction class
top5 = {rxn : top_k(rxn) for rxn in rxnClass}

In [ ]:
def tao(success, confidence, k):
    """
    This function computes the Tao score for the top-k outcomes from a single prediction. 
    
    Here, the probability is taken as the sum of all confidences associated with that prediction. 
    E.g., The top-5 probability of success for a reaction with 5 outcomes is the sum of the confidences for those 5 outcomes.
    """
    p = sum(confidence[0:k])
    if 'Y' in success[0:k]:
        return math.log2(2*p)
    else:
        return math.log2(2*(1-p))
        
        

In [ ]:
def get_tao(r):
    """
    This function calculates the total Tao score from the top-k outcomes for each reaction in a reaction class.
    """
    total_score = [0,0,0,0,0]
    for s,c in zip(outcome_dict[r]['success'],outcome_dict[r]['confidence']):
            for k in range(1,6):
                total_score[k-1] += tao(s,c,k)
    return total_score   

In [ ]:
#generate dictionary of top-k Tao scores divided according to reaction class
#values are lists of top-k scores in descending order, i.e., top-1, top-2, top-3, ...etc.
tao_scores = {r : get_tao(r) for r in rxnClass}

In [ ]:
#find outcomes containing extra elements or new elements absent in reactants
alchemy = {rxn : {'alchemy' : [], 'extra': []} for rxn in rxnClass}
err = ''
for i,(c,r,p) in enumerate(zip(more_outcomes['reaction class'],more_outcomes['reaction'],more_outcomes['product'])):
    reac = r.split('>')[0]
    el_r,el_p = [atom.GetSymbol() for atom in Chem.MolFromSmarts(reac).GetAtoms()],[atom.GetSymbol() for atom in Chem.MolFromSmarts(p).GetAtoms()]
    for element in el_p:
        result = [i, reac+'>>'+p, element]
        if element not in el_r and result not in alchemy[c]['alchemy']:
            alchemy[c]['alchemy'].append(result)
        elif el_p.count(element) > el_r.count(element) and result not in alchemy[c]['extra']:
            alchemy[c]['extra'].append(result)

In [ ]:
#find reactions with less than 5 outcomes
less_than_5_rxns = [[rxn,more_outcomes['reaction'].tolist().count(rxn)] for rxn in more_outcomes['reaction'] if more_outcomes['reaction'].tolist().count(rxn)<5]
[list(i) for i in set(map(tuple,less_than_5_rxns))]

In [ ]:
alchemy['organocopper']['alchemy']